## A simple standard finite element discretization

<!-- Loading a lot of secret LaTeX commands ... -->
$$
% \newcommand{\RR}{\mathbb{R}}
% \newcommand{\NN}{\mathbb{N}}
% \newcommand{\VV}{\mathbb{V}}
% \newcommand{\PP}{\mathbb{P}}
% \newcommand{\dGamma}{\,\mathrm{d} \Gamma}
% \newcommand{\dGammah}{\,\mathrm{d} \Gamma_h}
% \newcommand{\dx}{\,\mathrm{d}x}
% \newcommand{\dy}{\,\mathrm{d}y}
% \newcommand{\ds}{\,\mathrm{d}s}
% \newcommand{\dt}{\,\mathrm{d}t}
% \newcommand{\dS}{\,\mathrm{d}S}
% \newcommand{\dV}{\,\mathrm{d}V}
% \newcommand{\dX}{\,\mathrm{d}X}
% \newcommand{\dY}{\,\mathrm{d}Y}
% \newcommand{\dE}{\,\mathrm{d}E}
% \newcommand{\dK}{\,\mathrm{d}K}
% \newcommand{\dM}{\,\mathrm{d}M}
% \newcommand{\cd}{\mathrm{cd}}
% \newcommand{\onehalf}{\frac{1}{2}}
% \newcommand{\bfP}{\boldsymbol P}
% \newcommand{\bfx}{\boldsymbol x}
% \newcommand{\bfa}{\boldsymbol a}
% \newcommand{\bfu}{\boldsymbol u}
% \newcommand{\bfv}{\boldsymbol v}
% \newcommand{\bfe}{\boldsymbol e}
% \newcommand{\bfb}{\boldsymbol b}
% \newcommand{\bff}{\boldsymbol f}
% \newcommand{\bfp}{\boldsymbol p}
% \newcommand{\bft}{\boldsymbol t}
% \newcommand{\bfj}{\boldsymbol j}
% \newcommand{\bfB}{\boldsymbol B}
% \newcommand{\bfV}{\boldsymbol V}
% \newcommand{\bfE}{\boldsymbol E}
% \newcommand{\mcO}{\mathcal{O}}
% \newcommand{\mcT}{\mathcal{T}}
% \newcommand{\mrsd}{\mathrm{sd}}
% \newcommand{\foralls}{\forall\;}
% %\newcommand{\exists}{\exist\;}
% \newcommand{\hmax}{h_{\mathrm{max}}}
$$

## Example 1: Solution with an exponential boundary layer

The function $u_{\epsilon} = x\dfrac{1-e^{(y-1)/\epsilon}}{1-e^{-2/\epsilon}} $ solves the PDE
$$
\begin{align*}
-\epsilon \Delta u_{\epsilon} + \bfb \cdot \nabla u_{\epsilon} = f \quad \text{in } \Omega
\end{align*}
$$

with domain $\Omega = [-1,1]^2$, velocity field $\bfb = (0,1)$, right-hand side $f=0$, and Dirichlet boundary conditions given by $u_\epsilon$. 
This function is a typical example
of a solution which exhibits an exponential boundary layer at the outflow boundary. The width of this boundary layer is $\approx \mcO(\epsilon)$.
Now plot the solution for various $\epsilon$,, e.g. $\epsilon = 1, 10^{-1}, 10^{-3}, 10^{-5}$:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

def plot2D(X, Y, Z, title=""):
    # Define a new figure with given size an
    fig = plt.figure(figsize=(8, 6), dpi=100)
    # ax = fig.gca(projection='3d')
    ax = plt.axes(projection='3d')
    surf = ax.plot_surface(X, Y, Z,             
                            rstride=1, cstride=1, # Sampling rates for the x and y input data
                            cmap=cm.plasma)      # Use the colormap plasma
    
    # Set initial view angle
    ax.view_init(30, 225)
    
    # Set labels and show figure
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$u(x,y)$')
    ax.set_title(title)
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10)
    plt.show()

In [ ]:
# Example of solution with boundary layer
def u_ex(x, y, eps):
    return x*(1-np.exp((y-1)/eps))/(1-np.exp(-2/eps))
              
# Define subdivions in each direction
n = 50

# To define the grid we use "linspace" in each direction ...
xi = np.linspace(-1,1,n+1)
yi = np.linspace(-1,1,n+1)

x,y = np.meshgrid(xi,yi,sparse=True)

# Evaluate u on the grid. The output will be a 2 dimensional array 
# where U_ex_grid[i,j] = u_ex(x_i, y_j)
eps = 1e-3
U_ex_grid = u_ex(x,y, eps)

In [ ]:
%matplotlib widget
plot2D(x, y, U_ex_grid, title="$u(x,y)$")

## Example 2: Trying to solve the previous example with a standard finite element method

Now we illustrate what happens if you try to solve the example above with a standard finite element method, where we seek $u_h \in V_{h,g} = \PP^k_C(\mcT_h)
\cap H^1_g(\Omega)$ 
such that $\foralls v \in V_{h,0}$:

$$
a(u_h, v) := \epsilon(\nabla u_h, \nabla v)_{\Omega} + (b \cdot \nabla u_h, v_h)_{\Omega} + (c u_h, v_h)_{\Omega} = (f,v)_{\Omega} =: l(v)
$$

At the same time we use this example to introduce you to the finite element library [ngsolve](https://ngsolve.org), which has a large number of [Jupyter tutorials](https://ngsolve.org/docu/v6.2.2008/i-tutorials/index.html#).
The code below is basically compiled from 
[unit-1.1-poisson](https://ngsolve.org/docu/v6.2.2008/i-tutorials/unit-1.1-poisson/poisson.html) (solving Poisson with homogeneous Dirichlet b.c),
[unit-1.2-coefficient](https://ngsolve.org/docu/v6.2.2008/i-tutorials/unit-1.2-coefficient/coefficientfunction.html) (using coefficient functions to define $\bfb$), and
[unit-1.3-dirichlet](https://ngsolve.org/docu/v6.2.2008/i-tutorials/unit-1.3-dirichlet/dirichlet.html) (implemementation of non-homogeneous Dirichlet b.c.).

We will solve the problem for $\epsilon = 0.1$.

First, we start by importing the necessary modules for ngsolve (the finite element module)
and the ```SplineGeometry``` class which is part of the ```netgen``` module for mesh generation.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

Note that if you want to implement your code in pure Python script,
then you would rather import the old gui interface
```python
from netgen import gui
```
Also, to start up the python file with the netgen gui, you have to
execute the python file in the terminal via
```
netgen <PYTHONFILE.py>
```
rather then just typing `python <PYTHONFILE.py>`.

The next block generate first a domain description and then a mesh (unstructured) for
the doman $\Omega = [-1,1]^2$ with a given $h_{\mrm{max}}$.

Note that for the unit square domain, there exists a shortcut for the domain description and you can simply write 
```python 
from ngsolve import *
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
```

In [ ]:
shape = Rectangle(2,2).Face().Move((-1,-1, 0))
shape.edges.name="rectangle"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.25))
print(f"Boundary domains: {mesh.GetBoundaries()}")
Draw(mesh)

In [ ]:
# Define exact solution 
eps = 1.0e-3
u_ex = x*(1-exp((y-1)/eps))/(1-exp(-2/eps))
Draw(u_ex, mesh, name="u_ex")

# Define coefficients for PDE and exact solution
b = CoefficientFunction((0,1))
Draw(b, mesh, "b")

Let us redraw the function $u$ with different gui settings for ngsolve's webgui and with a different camera angle (defined
by the Euler rotation angles.)

In [ ]:
# Gui settings
gui_settings = settings = { "subdivision" : 1, "deformation": 1.0, "Colormap" : { "ncolors" : 32 } }
Draw (u_ex, mesh, name="u_ex", settings=gui_settings, euler_angles=[-90,0,60])
help(Draw)

First we have to define the discrete trial and test functions and their respectives spaces we want to use.

In [ ]:
# Define function spaces
p_order = 1
V = H1(mesh, order=p_order, dirichlet="rectangle")
u = V.TrialFunction()
v = V.TestFunction()
# Alternative shortcut
#u,v = V.TnT()

Next, we define bilinear form and linear form and assemble
them into a matrix and right-hand side vector, respectively.
Note that since $f=0$ in our case, we don't need
to assemble anything as the underlying vector associated
with ```l``` is initialized with a $0$ vector matching the dimension of the discrete function space. You can check
that easily by printing it
```python
print(l.vec)
```

In [ ]:
a = BilinearForm(V) 
a += eps*grad(u)*grad(v)*dx
a += b*grad(u)*v*dx
a.Assemble()

l = LinearForm(V)
# If right-hand side is 0, we don't need to do anything
#l += 0*v*dx
# l.Assemble()

Now we solve the overall system. Note that we need 
to incorporate the inhomogeneous Dirichlet boundary data,
see [unit-1.3-dirichlet](https://ngsolve.org/docu/v6.2.2008/i-tutorials/unit-1.3-dirichlet/dirichlet.html) for a detailed explanation.

In [ ]:
# Set boundary data
u_h = GridFunction(V)
u_h.Set(u_ex, BND)
Draw(u_h, name="u_D", settings=gui_settings)

# Solve final system
r = l.vec.CreateVector()
r.data = l.vec - a.mat*u_h.vec
print(type(r.data))
u_h.vec.data += a.mat.Inverse(freedofs=V.FreeDofs())*r
Draw(u_h, name="u_h", settings=gui_settings)

Finally, we write out the solution as vtk file for visualization with paraview.

In [ ]:
vtk = VTKOutput(ma=mesh,coefs=[u_ex, u_h],
                names=["u_ex","u_h"],
                filename="adr-example",
                subdivision=1)
vtk.Do()